# Car Price Prediction::

Download dataset from this link:

https://www.kaggle.com/hellbuoy/car-price-prediction

# Problem Statement::

A Chinese automobile company Geely Auto aspires to enter the US market by setting up their manufacturing unit there and producing cars locally to give competition to their US and European counterparts.

They have contracted an automobile consulting company to understand the factors on which the pricing of cars depends. Specifically, they want to understand the factors affecting the pricing of cars in the American market, since those may be very different from the Chinese market. The company wants to know:

Which variables are significant in predicting the price of a car
How well those variables describe the price of a car
Based on various market surveys, the consulting firm has gathered a large data set of different types of cars across the America market.

# task::
We are required to model the price of cars with the available independent variables. It will be used by the management to understand how exactly the prices vary with the independent variables. They can accordingly manipulate the design of the cars, the business strategy etc. to meet certain price levels. Further, the model will be a good way for management to understand the pricing dynamics of a new market.

# WORKFLOW ::

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

4.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

5.Compilation Step (Note : Its a Regression problem , select loss , metrics according to it)
6.Train the Model with Epochs (100) and validate it

7.If the model gets overfit tune your model by changing the units , No. of layers , activation function , epochs , add dropout layer or add Regularizer according to the need .

8.Evaluation Step

9.Prediction

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [131]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [106]:
df = pd.read_csv("/content/drive/MyDrive/Deep Learning Assignments/DLAssignments/dataset/CarPrice_Assignment.csv")

In [21]:
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [22]:
df.isnull().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [26]:
le = LabelEncoder()
df["fueltype"] = le.fit_transform(df["fueltype"])
df["aspiration"]= le.fit_transform(df["aspiration"])
df["carbody"]= le.fit_transform(df["carbody"])
df["drivewheel"]= le.fit_transform(df["drivewheel"])
df["enginelocation"]= le.fit_transform(df["enginelocation"])
df["doornumber"]= df["doornumber"].map({"two":2, "four":4})

In [34]:
corelation_matrix = df.corr()
corelation_matrix

,car_ID,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
car_ID,1.000000,-0.151621,-0.125568,0.067729,0.190352,0.098303,0.051406,0.051483,0.129729,0.170636,0.052387,0.255960,0.071962,-0.033930,0.260064,-0.160824,0.150276,-0.015006,-0.203789,0.015940,0.011255,-0.109093
symboling,-0.151621,1.000000,0.194311,-0.059866,-0.664073,-0.596135,-0.041671,0.212471,-0.531954,-0.357612,-0.232919,-0.541038,-0.227691,-0.105790,-0.130051,-0.008735,-0.178515,0.070873,0.273606,-0.035823,0.034606,-0.079978
fueltype,-0.125568,0.194311,1.000000,-0.401397,-0.191491,-0.147853,-0.132257,0.040070,-0.308346,-0.212679,-0.233880,-0.284631,-0.217275,-0.069594,-0.054451,-0.241829,-0.984356,0.163926,0.476883,-0.255963,-0.191392,-0.105679
aspiration,0.067729,-0.059866,-0.401397,1.000000,0.031792,0.063028,0.066465,-0.057191,0.257611,0.234539,0.300567,0.087311,0.324902,0.108217,0.212614,0.222982,0.295541,0.241685,-0.183383,-0.202362,-0.254416,0.177926
doornumber,0.190352,-0.664073,-0.191491,0.031792,1.000000,0.680358,-0.098954,-0.137757,0.447357,0.398568,0.207168,0.552208,0.197379,0.020742,0.119258,-0.011082,0.177888,-0.126947,-0.247668,-0.012417,-0.036330,0.031835
carbody,0.098303,-0.596135,-0.147853,0.063028,0.680358,1.000000,-0.155745,-0.277009,0.401362,0.334433,0.131710,0.568534,0.128467,-0.073352,0.010549,-0.015325,0.136243,-0.153928,-0.109643,0.031697,-0.007170,-0.083976
drivewheel,0.051406,-0.041671,-0.132257,0.066465,-0.098954,-0.155745,1.000000,0.147865,0.459745,0.485649,0.470751,-0.019719,0.575111,0.524307,0.481827,0.071591,0.127479,0.518686,-0.039417,-0.449581,-0.452220,0.577992
enginelocation,0.051483,0.212471,0.040070,-0.057191,-0.137757,-0.277009,0.147865,1.000000,-0.187790,-0.050989,-0.051698,-0.106234,0.050468,0.196826,0.185042,-0.138455,-0.019762,0.317839,0.198461,-0.153487,-0.102026,0.324973
wheelbase,0.129729,-0.531954,-0.308346,0.257611,0.447357,0.401362,0.459745,-0.187790,1.000000,0.874587,0.795144,0.589435,0.776386,0.569329,0.488750,0.160959,0.249786,0.353294,-0.360469,-0.470414,-0.544082,0.577816
carlength,0.170636,-0.357612,-0.212679,0.234539,0.398568,0.334433,0.485649,-0.050989,0.874587,1.000000,0.841118,0.491029,0.877728,0.683360,0.606454,0.129533,0.158414,0.552623,-0.287242,-0.670909,-0.704662,0.682920


In [41]:
useful_features =df[["horsepower","boreratio","carlength","carwidth","curbweight","enginesize",
                   "price","wheelbase","enginelocation","drivewheel"]]  

In [50]:
x = (useful_features.loc[:, useful_features.columns !="price"])
y = (useful_features.loc[:, useful_features.columns =="price"])

In [51]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.30, random_state=50)

In [145]:
model = Sequential()
def myModel():
    
    model.add(Dense(10, input_shape=(9,)))
    model.add(Dense(8,activation="relu"))
    model.add(Dense(6, activation="relu"))
    return model

In [146]:
myModel()

In [147]:
model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 10)                100       
_________________________________________________________________
dense_56 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_57 (Dense)             (None, 6)                 54        
Total params: 242
Trainable params: 242
Non-trainable params: 0
_________________________________________________________________


In [141]:
model.compile(optimizer='Adam',loss='mse', metrics=['mae','accuracy'])

In [117]:
model.fit(x_train, y_train, validation_split=0.20, epochs=100)

Epoch 1/100
4/4 [==============================] - 1s 76ms/step - loss: 178723187.2000 - mae: 10631.1012 - accuracy: 0.0000e+00 - val_loss: 166913008.0000 - val_mae: 10201.6982 - val_accuracy: 0.0000e+00
Epoch 2/100
4/4 [==============================] - 0s 17ms/step - loss: 173761065.6000 - mae: 10566.8594 - accuracy: 0.0000e+00 - val_loss: 166872224.0000 - val_mae: 10191.4688 - val_accuracy: 0.0000e+00
Epoch 3/100
4/4 [==============================] - 0s 16ms/step - loss: 169187100.8000 - mae: 10335.7594 - accuracy: 0.0000e+00 - val_loss: 166868192.0000 - val_mae: 10191.5898 - val_accuracy: 0.0000e+00
Epoch 4/100
4/4 [==============================] - 0s 15ms/step - loss: 170393392.0000 - mae: 10448.1645 - accuracy: 0.0000e+00 - val_loss: 166870432.0000 - val_mae: 10193.6133 - val_accuracy: 0.0000e+00
Epoch 5/100
4/4 [==============================] - 0s 16ms/step - loss: 196757440.0000 - mae: 10994.3930 - accuracy: 0.0000e+00 - val_loss: 166892384.0000 - val_mae: 10200.8633 - val_a

In [118]:
predict = model.predict(x_test)

In [119]:
predict

array([[17978.93  ,     0.    , 18020.777 ,     0.    ,     0.    ,
            0.    ],
       [15039.905 ,     0.    , 15026.18  ,     0.    ,     0.    ,
            0.    ],
       [15295.595 ,     0.    , 15267.41  ,     0.    ,     0.    ,
            0.    ],
       [22352.008 ,     0.    , 22211.219 ,     0.    ,     0.    ,
            0.    ],
       [13967.275 ,     0.    , 13982.68  ,     0.    ,     0.    ,
            0.    ],
       [10602.573 ,     0.    , 10637.297 ,     0.    ,     0.    ,
            0.    ],
       [16612.414 ,     0.    , 16641.693 ,     0.    ,     0.    ,
            0.    ],
       [17182.223 ,     0.    , 17186.643 ,     0.    ,     0.    ,
            0.    ],
       [12814.818 ,     0.    , 12836.809 ,     0.    ,     0.    ,
            0.    ],
       [10531.501 ,     0.    , 10562.469 ,     0.    ,     0.    ,
            0.    ],
       [15996.759 ,     0.    , 15961.334 ,     0.    ,     0.    ,
            0.    ],
       [12179.68  ,  

In [120]:
model.evaluate(x_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 167590784.0000 - mae: 10202.3945 - accuracy: 0.0000e+00


[167590784.0, 10202.39453125, 0.0]

In [121]:
model.evaluate(x_train, y_train)

5/5 [==============================] - 0s 3ms/step - loss: 172277136.0000 - mae: 10349.2852 - accuracy: 0.0000e+00


[172277136.0, 10349.28515625, 0.0]

In [122]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 10)                100       
_________________________________________________________________
dense_46 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_47 (Dense)             (None, 6)                 54        
Total params: 242
Trainable params: 242
Non-trainable params: 0
_________________________________________________________________
